In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='D2Coding')
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

In [35]:
sales_df = pd.read_csv('../data/Sales_data.csv', low_memory=False)
display(sales_df.head())
display(sales_df.shape)

,주문번호,구매일,배송시작일,배송완료일,구매시월령(수정),고객번호,구매금액,결제금액,물품대분류,상품명
0,201905052342970,2019-05-06,2019-05-07,2019-05-11,4.0,201812310749735,25800,16314,더블하트,더블하트 SofTouch 모유실감 젖꼭지 M (2입)
1,202002203424450,2020-02-20,2020-02-21,2020-02-23,14.0,201812310749735,51200,37769,팬티,NEW 하기스 맥스드라이 팬티 5단계 남아 80매(팬티형)
2,201902282095385,2019-02-28,2019-03-01,2019-03-05,NaN,201812310749770,5610,3800,FMC,크리넥스？키즈용？화장지？3겹？27m 8롤
3,201905132367003,2019-05-13,2019-05-14,2019-05-16,66.0,201812310749771,59520,47922,기저귀,하기스 네이처메이드 밤부 3단계 여아 144매(밴드형)
4,201905092356247,2019-05-10,2019-05-11,2019-05-13,5.0,201812310749774,50640,39788,기저귀,하기스 에어솔솔 썸머기저귀 3단계 공용 144매(밴드형)


(199999, 10)

In [36]:
mask = np.any(sales_df.isna(), axis=1)
print(f'결측치의 수: {mask.sum()}')

df = sales_df[~mask]
display(df.head())
display(df.shape)

결측치의 수: 31857


,주문번호,구매일,배송시작일,배송완료일,구매시월령(수정),고객번호,구매금액,결제금액,물품대분류,상품명
0,201905052342970,2019-05-06,2019-05-07,2019-05-11,4.0,201812310749735,25800,16314,더블하트,더블하트 SofTouch 모유실감 젖꼭지 M (2입)
1,202002203424450,2020-02-20,2020-02-21,2020-02-23,14.0,201812310749735,51200,37769,팬티,NEW 하기스 맥스드라이 팬티 5단계 남아 80매(팬티형)
3,201905132367003,2019-05-13,2019-05-14,2019-05-16,66.0,201812310749771,59520,47922,기저귀,하기스 네이처메이드 밤부 3단계 여아 144매(밴드형)
4,201905092356247,2019-05-10,2019-05-11,2019-05-13,5.0,201812310749774,50640,39788,기저귀,하기스 에어솔솔 썸머기저귀 3단계 공용 144매(밴드형)
5,201907282649358,2019-07-29,2019-07-30,2019-08-02,8.0,201812310749774,101280,76744,기저귀,[2019년형] 하기스 에어솔솔 썸머기저귀 3단계 공용 144매(밴드형)


(168142, 10)

In [37]:
df = df.copy()
df['구매일'] = pd.to_datetime(df['구매일'], format='%Y-%m-%d')
df['배송시작일'] = pd.to_datetime(df['배송시작일'], format='%Y-%m-%d')
df['배송완료일'] = pd.to_datetime(df['배송완료일'], format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168142 entries, 0 to 199996
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   주문번호       168142 non-null  object        
 1   구매일        168142 non-null  datetime64[ns]
 2   배송시작일      168142 non-null  datetime64[ns]
 3   배송완료일      168142 non-null  datetime64[ns]
 4   구매시월령(수정)  168142 non-null  float64       
 5   고객번호       168142 non-null  object        
 6   구매금액       168142 non-null  int64         
 7   결제금액       168142 non-null  int64         
 8   물품대분류      168142 non-null  object        
 9   상품명        168142 non-null  object        
dtypes: datetime64[ns](3), float64(1), int64(2), object(4)
memory usage: 14.1+ MB


## RFM 구하기

In [96]:
"""
column: [고객번호, 첫 구매일, 마지막 구매일, 마지막 구매 후 경과 일자, 구매 횟수, 평균 구매 주기, 결제금액, 구매금액]

Recency = 고객 ID 별 마지막 구매
"""

# Recency: 고객 ID 별 마지막 구매
sorted_day = df.sort_values(['고객번호', '구매일'])[['고객번호', '구매일']]
first_day = sorted_day.drop_duplicates(subset='고객번호', keep='first').rename(columns={'구매일': '첫 구매일'})

# Frequency: 고객 ID 별 구매 횟수
last_day = sorted_day.drop_duplicates(subset='고객번호', keep='last').rename(columns={'구매일': '마지막 구매일'})
df_rfm = pd.merge(first_day, last_day, on='고객번호', how='inner')
df_rfm['마지막 구매 후 경과 일자'] = df_rfm['마지막 구매일'].max() - df_rfm['마지막 구매일']
df_rfm['마지막 구매 후 경과 일자'] = df_rfm['마지막 구매 후 경과 일자'].dt.days
df_rfm['구매 횟수'] = df.groupby(by='고객번호', as_index=False).count().iloc[:, 1]
df_rfm['평균 구매 주기'] = df_rfm['마지막 구매일'] - df_rfm['첫 구매일']
df_rfm['평균 구매 주기'] = df_rfm['평균 구매 주기'].dt.days

# Frequency: 고객 ID 별 평균 구매 주기
df_temp = df_rfm[['고객번호', '구매 횟수', '평균 구매 주기']]
df_temp = df_temp[df_temp['평균 구매 주기'] != 0]
df_temp['평균 구매 주기'] = df_temp['평균 구매 주기'] / (df_temp['구매 횟수'] - 1)
df_rfm = pd.merge(df_rfm.drop('평균 구매 주기', axis=1), df_temp[['고객번호', '평균 구매 주기']], on='고객번호', how='left')
df_rfm = df_rfm.replace(np.NaN, 0)
df_rfm['평균 구매 주기'] = df_rfm['평균 구매 주기'].astype(int)

# Monetary: 고객 ID 별 결제금액 합산
sorted_sum_money = df[['고객번호', '결제금액', '구매금액']].sort_values('고객번호').groupby(by='고객번호', as_index=False).sum()
df_rfm = pd.merge(df_rfm, sorted_sum_money, on='고객번호', how='inner')

df_rfm.head()

,고객번호,첫 구매일,마지막 구매일,마지막 구매 후 경과 일자,구매 횟수,평균 구매 주기,결제금액,구매금액
0,201812310749735,2019-05-06,2020-02-20,169,2,290,54083,77000
1,201812310749771,2019-05-13,2019-05-13,452,1,0,47922,59520
2,201812310749774,2019-05-10,2019-07-29,375,2,80,116532,151920
3,201812310749780,2019-01-30,2019-01-30,555,1,0,19900,27920
4,201812310749784,2019-04-29,2020-07-23,15,8,64,166877,270840


In [98]:
df_rfm.to_csv('../data/rfm.csv', encoding='euc-kr', index=False)
df_rfm

,고객번호,첫 구매일,마지막 구매일,마지막 구매 후 경과 일자,구매 횟수,평균 구매 주기,결제금액,구매금액
0,201812310749735,2019-05-06,2020-02-20,169,2,290,54083,77000
1,201812310749771,2019-05-13,2019-05-13,452,1,0,47922,59520
2,201812310749774,2019-05-10,2019-07-29,375,2,80,116532,151920
3,201812310749780,2019-01-30,2019-01-30,555,1,0,19900,27920
4,201812310749784,2019-04-29,2020-07-23,15,8,64,166877,270840
...,...,...,...,...,...,...,...,...
65727,20200708231025349606,2020-07-09,2020-07-09,29,1,0,28102,49900
65728,20200721194641507158,2020-08-01,2020-08-01,6,1,0,25480,25480
65729,20200724095212627384,2020-07-24,2020-07-24,14,1,0,0,0
65730,20200724113017315991,2020-07-24,2020-07-24,14,1,0,9240,11700
